# 1. 前言

## 1.1 项目背景

苹果是世界上最重要的温带水果作物之一。叶病对苹果种植园的整体生产力和质量构成重大威胁。目前苹果种植园的植物疾病诊断的过程主要依赖于人工，既费时又昂贵。

虽然基于计算机视觉的模型在植物疾病识别方面已显示出希望，但仍有一些限制需要解决。不同苹果品种或栽培中的新品种中单一疾病的视觉症状差异很大，是计算机基于视觉的疾病识别的主要挑战。这些变化产生于自然和图像捕捉环境的差异，例如叶色和叶形、受感染组织的年龄、不均匀的图像背景以及成像过程中不同的光照等。

植物病理学2020-FGVC7挑战竞赛有3651个RGB图像的苹果叶病的试点数据集。对于植物病理学 2021-FGVC8，我们显著增加了叶病图像的数量，并增加了额外的疾病类别。今年的数据集包含大约 23000 张苹果叶病的高质量 RGB 图像，包括一个大型专家注释的疾病数据集。此数据集通过表示在不同成熟阶段和一天中不同时间在不同焦距摄像机设置下拍摄的叶子图像的非同质背景来反映真实的字段场景。

> 竞赛地址：https://www.kaggle.com/c/plant-pathology-2021-fgvc8  

虽然目前竞赛已经结束了，但该组织会每年举办一次，明年参加或者参考本文思路用Swin Transformer刷其他竞赛榜单！

## 1.2 任务目标

竞赛的主要目标是开发基于机器学习的模型，将特定的叶子图像从测试数据集精确分类到特定的疾病类别，并在单个叶子图像上从多种疾病症状中识别出单个疾病症状的单个疾病。

## 1.3 Swin Transformer

![](https://ai-studio-static-online.cdn.bcebos.com/5f2e1cf95ec44b41a1127338a31920da08127f12d10248f09a621c0c2d18019f)

Swin Transformer是一个新的Transformer模型，使用了CNN结构设计中的一些理念（降采样、局部dependency、TSM）来重新设计Transformer，目前已经屠榜各大深度学习视觉任务榜单。

详细介绍可以看仰世大佬的文章 -> https://aistudio.baidu.com/aistudio/projectdetail/1796427?channel=0&channelType=0&shared=1

本文主要是通过一个竞赛案例，来展示Swin Transformer究竟有多牛逼！

# 2. 数据集预处理

## 2.1 解压数据集

In [ ]:
!unzip data/data100996/Plant_Pathology_2021.zip -d data > /dev/null 2>&1

## 2.2 生成训练文件列表

### 2.2.1 统计数据分布

In [ ]:
import pandas as pd

data_csv = pd.read_csv('data/train.csv',header=0)
labels_counts = data_csv['labels'].value_counts()
labels_counts

从上面可以看出，数据集包含12种叶病类型，并且各种类型图像的数量明显分布不均匀，后面应该要考虑下数据增强，有时间的朋友可以试试修改Loss增加权重。

### 2.2.2 生成列表

In [ ]:
labels_list = list(labels_counts.keys())
#生成labels.txt
for l in range(len(labels_list)):
    label = labels_list[l].replace(' ','_')
    with open('./data/label.txt','a+') as lb:
        lb.write('%d %s\n'%(l,label))
print('生成data/labels.txt完成。')

In [ ]:
ratio = 0.7 #训练集和验证集比例
for c in range(len(labels_list)):
    class_img = data_csv[data_csv['labels']==labels_list[c]]
    class_img_num = len(class_img)
    train_data,val_data = class_img.iloc[0:int(class_img_num*ratio)],class_img.iloc[int(class_img_num*ratio):class_img_num]
    for train in range(len(train_data)):
        with open('data/train.txt','a+') as t:
            t.write('%s %d\n'%(train_data.iloc[train]['image'],c))
    for val in range(len(val_data)):
        with open('data/val.txt','a+') as v:
            v.write('%s %d\n'%(val_data.iloc[val]['image'],c))

print('生成训练集/测试集完成。')

## 2.3 初步观察数据特征，制定优化策略

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image
import os

#plt.imshow(Image.open('data/train_images/80070f7fb5e2ccaa.jpg'))
sample_data = train_data.sample(n=25)
plt.figure(figsize=(15,10))
for i in range(25):
    img_sample,label_sample = sample_data.iloc[i]['image'],sample_data.iloc[i]['labels']
    plt.subplot(5,5,i+1)
    plt.imshow(Image.open(os.path.join('data/train_images',img_sample)))
    plt.title(label_sample)
    plt.axis('off')

由上图的数据可以看出，数据集的叶片位置大致都位于图像中央，光线条件几乎一样，因此不需要亮度对比度等图像增强，只需要关注图像中间部分。我在PaddleClas自定义了一个Centercrop方法,调用paddle.vision.transforms中的center_crop函数。在配置文件中，先对训练图像进行Resize，以减少内存消耗，再通过Centercrop裁剪图片，减少图像背景其他干扰信息。

# 3. 训练模型

## 3.1 安装依赖

In [ ]:
!pip install -r PaddleClas/requirements.txt

## 3.2 配置文件(train.yaml)

> 由于图像识别的主体主要分布在图像的中央，背景存在干扰信息，因此我修改了paddleclas的transforms方法，移植了paddle.vision.transforms的centercrop方法，修改的文件路径是 PaddleClas/ppcls/data/preprocess/ops/operators.py

In [ ]:
from paddle.vision.transforms import center_crop

class Centercrop(object):
    def __init__(self, size, keys=None):
        if type(size) is int:
            self.size = (size, size)  # (h, w)
        else:
            self.size = size

    def __call__(self, img):
        return center_crop(img, self.size)

In [ ]:
# 全局配置
Global:
  pretrained_model: SwinTransformer_base_patch4_window7_224_22kto1k_pretrained #预训练模型
  output_dir: ./output/ #模型和日志信息保存文件夹
  device: gpu #使用GPU进行训练
  save_interval: 30 #每30个inter保存一个checkpoint
  eval_during_train: True #模型训练过程中评估
  eval_interval: 1 #每一轮训练后评估一次
  epochs: 120 #总迭代次数
  print_batch_step: 100 #每100个step打印一条进度记录
  use_visualdl: True # 开启VisuaDL可视化

# 网络模型
Arch:
  name: SwinTransformer_base_patch4_window7_224
  class_num: 12 #分类数量
 
# 损失函数使用CrossEntropy Loss
Loss:
  Train:
    - CELoss:
        weight: 1.0
  Eval:
    - CELoss:
        weight: 1.0

#优化器
Optimizer:
  name: Momentum
  momentum: 0.9
  lr:
    name: Cosine #使用cosine替代原配置文件中的piecewise
    learning_rate: 0.001 #学习率
  regularizer:
    name: 'L2'
    coeff: 0.0001


# Dataloader
DataLoader:
  Train:
    dataset:
      name: ImageNetDataset
      image_root: data/train_images #训练图像保存的文件夹
      cls_label_path: data/train.txt #训练图像路径列表
      transform_ops:
        - DecodeImage:
            to_rgb: True
            channel_first: False
        - ResizeImage: #根据硬件配置先把图像resize到合适的大小
            size: 448
        - Centercrop: #加入centercrop排除背景信息过多干扰
            size: 224
        - RandFlipImage:
            flip_code: 1
        - NormalizeImage:
            scale: 1.0/255.0
            mean: [0.485, 0.456, 0.406]
            std: [0.229, 0.224, 0.225]
            order: ''

    sampler:
      name: DistributedBatchSampler
      batch_size: 8 #batchsize
      drop_last: False
      shuffle: True
    loader:
      num_workers: 0
      use_shared_memory: True

  Eval:
    dataset: 
      name: ImageNetDataset
      image_root: data/train_images #评估图像保存的文件夹
      cls_label_path: data/val.txt #评估图像路径列表
      transform_ops:
        - DecodeImage:
            to_rgb: True
            channel_first: False
        - ResizeImage:
            resize_short: 256
        - CropImage:
            size: 224
        - NormalizeImage:
            scale: 1.0/255.0
            mean: [0.485, 0.456, 0.406]
            std: [0.229, 0.224, 0.225]
            order: ''
    sampler:
      name: DistributedBatchSampler
      batch_size: 8
      drop_last: False
      shuffle: False
    loader:
      num_workers: 0
      use_shared_memory: True

Infer:
  transforms:
    - DecodeImage:
        to_rgb: True
        channel_first: False
    - ResizeImage:
        resize_short: 256
    - CropImage:
        size: 224
    - NormalizeImage:
        scale: 1.0/255.0
        mean: [0.485, 0.456, 0.406]
        std: [0.229, 0.224, 0.225]
        order: ''
    - ToCHWImage:
  PostProcess:
    name: Topk
    topk: 5
    class_id_map_file: data/label.txt  #Label列表

Metric:
  Train:
    - TopkAcc:
        topk: [1, 3]
  Eval:
    - TopkAcc:
        topk: [1, 3] #输出前1和前3的预测结果准确率

## 3.3 开始训练

### 3.3.1 下载预训练模型

In [ ]:
!wget https://paddle-imagenet-models-name.bj.bcebos.com/dygraph/SwinTransformer_base_patch4_window7_224_22kto1k_pretrained.pdparams

### 3.3.2 第一次训练

In [ ]:
!python PaddleClas/tools/train.py -c train.yaml

### 3.3.3 中断恢复训练

In [ ]:
!python PaddleClas/tools/train.py \
    -c train.yaml \
    -o Global.checkpoints="./output/MobileNetV3_large_x1_0/epoch_30"

### 3.3.4 观察训练过程

![](https://ai-studio-static-online.cdn.bcebos.com/966f86a0d0424c04a886883ca5f4f7d6a8b3d6c687a9441b8d2c94d69abe4494)

> 通过VisualDL可以看出，Swin Transfromer训练拟合的速度很快，第一轮训练准确率即超过0.8，并随着迭代的增加不断上升，算力所限，我只训练了6个迭代，建议算力和时间充足的朋友可以尝试多训练几轮，精度应该还能显著提高。

## 3.4 模型评估

In [ ]:
!python PaddleClas/tools/eval.py \
    -c train.yaml \
    -o Global.pretrained_model=output/SwinTransformer_base_patch4_window7_224/best_model

![](https://ai-studio-static-online.cdn.bcebos.com/16ae471914e54537a0914dfe3cc21bd8bfaf8296310c461fa6fc8f87fcf3f36b)

> 6轮迭代的精度就已经达到了 0.87328！

## 3.5 模型预测

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image
import os

plt.figure(figsize=(15,10))
test_img_list = os.listdir('data/test_images/')
for i in range(3):
    plt.subplot(1,3,i+1)
    plt.imshow(Image.open(os.path.join('data/test_images/',test_img_list[i])))
    plt.axis('off')

In [ ]:
!python PaddleClas/tools/infer.py \
    -c train.yaml \
    -o Infer.infer_imgs=data/test_images \
    -o Global.pretrained_model=output/SwinTransformer_base_patch4_window7_224/best_model

# 4.小总结

本文以苹果叶面病害任务演示了Swin Transformer训练预测的过程，跑了一轮发现Swin Transformer在拟合速度和精度方面对于主流的CNN模型有明显的优势，几轮迭代就能得到较好的结果，但其缺点也是很突出，一是消耗硬件性能，就算是aistudio的Tesla V100 32G的环境，也要在batchsize极小的情况下才能勉强跑通；二是训练速度明显慢于CNN网络，因此，在实际任务中，需要平衡速度和精度，慎重决定是否采用Transfomer方案。